[![Kaggle link](https://img.shields.io/badge/изначальный%20датасет-Открыть-orange?style=social&logo=kaggle)](https://www.kaggle.com/datasets/leomartinelli/bullying-in-schools)


# §1. Зависимости

In [317]:
import pandas as pd
from tabulate import tabulate
import numpy as np
%matplotlib inline 

## §2. Описание изначального датасета

In [318]:
data.shape

(56981, 18)

In [319]:
df_csv_path = '/Users/blacksnow/bmstu/ММО/Лабораторные работы/datasets/Bullying_2018_raw.csv'
bullying_df = pd.read_csv(df_csv_path) 

In [320]:


bull_desc = [
    ['Номер ответа','-'],
    ['В школе (за последние 12 месяцев)','Категориальный'], # - №2
    ['Вне школы (за последние 12 месяцев)','Категориальный'], # - №3
    ['Кибербуллинг (последние 12 месяцев?)','Категориальный'], # - №3
    ['Возраст респондента','Числовой'], # - №4
    ['Пол респондента','Категориальный'], # - №5
    ['Количество физических атак','Числовой'], # - №6
    ['Количество драк','Числовой'], # # - №7
    ['Ощущение одинокости','Категориальный'], # # - №8
    ['Количество близких друзей','Числовой'], # - №9
    ['Количество дней пропусков школы без разрешения','Числовой '], # - №10
    ['Доброта и отзывчивость окружающих','Категориальный'], # - №11
    ['Частота проблем взаимопонимания с родителями','Категориальный'], # - №12
    ['Ощущение одинокости большую части (частота)','Категориальный'], # # - №13
    ['Пропускали ли вы школу без разрешения (да/нет)','Категориальный'], # - №14
    ['Масса вашего тела в школе была ниже нормы?','Категориальный'], # - №15
    ['Масса вашего тела в школе была выше нормы?','Категориальный'], # - №16
    ['Вы страдали ожирением?','Категориальный'], # - №17
]


## §5. Проверка на пропуски изначального датасета

In [344]:
bullying_df.columns

Index(['record', 'Bullied_on_school_property_in_past_12_months',
       'Bullied_not_on_school_property_in_past_12_months',
       'Cyber_bullied_in_past_12_months', 'Custom_Age', 'Sex',
       'Physically_attacked', 'Physical_fighting', 'Felt_lonely',
       'Close_friends', 'Miss_school_no_permission',
       'Other_students_kind_and_helpful', 'Parents_understand_problems',
       'Most_of_the_time_or_always_felt_lonely',
       'Missed_classes_or_school_without_permission', 'Were_underweight',
       'Were_overweight', 'Were_obese'],
      dtype='object')

In [373]:
df_variants = []

for col in bullying_df.columns:
    variants = bullying_df[col].unique()
    if len(variants)<10:
        print(f'### Варианты ответа на ```{col}```',end='\n\n')
        print(tabulate([f'{variants[x]}' for x in range(0,len(variants))],tablefmt='github'),end='\n\n')            
    # df_variants.append(variants)


### Варианты ответа на ```Bullied_on_school_property_in_past_12_months```

|---|---|---|
| Y | e | s |
| N | o |   |
|   |   |   |

### Варианты ответа на ```Bullied_not_on_school_property_in_past_12_months```

|---|---|---|
| Y | e | s |
| N | o |   |
|   |   |   |

### Варианты ответа на ```Cyber_bullied_in_past_12_months```

|---|---|---|
|   |   |   |
| N | o |   |
| Y | e | s |

### Варианты ответа на ```Custom_Age```

|---|---|--|---|---|---|---|---|--|---|---|---|--|---|---|--|---|---|---|---|---|---|---|
| 1 | 3 |  | y | e | a | r | s |  | o | l | d |  |   |   |  |   |   |   |   |   |   |   |
| 1 | 4 |  | y | e | a | r | s |  | o | l | d |  |   |   |  |   |   |   |   |   |   |   |
| 1 | 6 |  | y | e | a | r | s |  | o | l | d |  |   |   |  |   |   |   |   |   |   |   |
| 1 | 2 |  | y | e | a | r | s |  | o | l | d |  |   |   |  |   |   |   |   |   |   |   |
| 1 | 5 |  | y | e | a | r | s |  | o | l | d |  |   |   |  |   |   |   |   |   |   |   |
| 1 | 1 |  | y | e | a | r | s |

## §3. Статистика по датасету

In [ ]:

def stats(data=bullying_df, desc=bull_desc, headers=('Номер','Название','Описание','Кат.','Числ.','Кол-во вариаций')):
    headers=('Номер','dtype','Описание','Тип','Кол-во пропусков','Кол-во вариаций')
    table = np.arange(data.shape[1]*len(headers))
    table = table.reshape(len(headers),data.shape[1])

    table = np.array([
                [x for x in range(1,19)],
                [d for d in data.dtypes],
                [a[0] for a in desc],
                [a[1] for a in desc],
                [data[col].isna().sum() for col in data.columns],
                [data[col].nunique() for col in data.columns]])

    table = table.transpose()

    print(
        tabulate(
            [data.shape],
            headers=('Кол-во строк', 'Кол-во столбцов'),
            tablefmt="github"),
        end='\n\n\n'
        )
    
    print(
        tabulate(
            table,
            headers=headers,
            tablefmt='github'
        ),
        end='\n\n\n'
        )

stats()

## §4. Нормализация данных

In [ ]:
def normalized_source(df=bullying_df):
    return df

stats(data=normalized_source())
